# Importing libraries

In [1]:
import cv2
import numpy as np

# Function to get the area of contours

In [2]:
def get_contour_areas(contours):
    # returns the areas of all contours as list
    all_areas = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        all_areas.append(area)
    return all_areas

# Importing image and pre-processing

In [3]:
img = cv2.imread('doc.jpg')

img2 = img.copy()

gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

blurred_image = cv2.GaussianBlur(img, (5, 5), 0)
                 
canny = cv2.Canny(blurred_image, 100 ,  200) 

# Contours
* Getting all the contours with their areas
* Sorting the contours in descending order and drawing only the first(biggest) contour on the image
* Getting the coordinates and height, width of our contour(x,y,w,h)

In [4]:
contours, hierarchy = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

all_areas = get_contour_areas(contours)

sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)     

for c in sorted_contours:
    #cv2.drawContours(img, [c], -1, (0,255,0), 2)
    #cv2.drawContours(gray_img, [c], -1, (0,255,0), 2)
    accuracy = 0.01 * cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, accuracy, True)
    cv2.drawContours(img, [approx], 0, (0, 0, 0), 2)
    x,y,w,h = cv2.boundingRect(c)
    break

# Getting the corners of the contour
* Converting to HSV color format
* Getting a mask of the black contour we drew on the image 
* Then we get the corners from the mask(max. of 4 corners)

In [5]:
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

lower_range = np.array([0,0,0])                   
upper_range = np.array([0, 0,0])
 
mask = cv2.inRange(hsv_img, lower_range, upper_range)
 
corners = cv2.goodFeaturesToTrack(mask, 4, 0.5, 80) 
corners = np.int0(corners)                              

# Making circles on the corners and numbering them for reference

In [6]:
points = []
i=1
font = cv2.FONT_HERSHEY_SIMPLEX

for corner in corners:
    
    x, y = corner.ravel()                               
    cv2.circle(img, (x, y), 4, (0, 0, 255), -1)
    cv2.putText(img, str(i), (x, y), font, 1, (0, 255, 255), 3, cv2.LINE_AA) 
    points.append((x,y))
    i+=1

# Arranging the coordinates prior to perspective transform
* The corners used to perform perspective transform are in the order of (top left,top right,bottom left,bottom right)
* The corners we detect from the contour will not necessarily be in this order
* Hence we arrange them
* If x and y are the coordinates of a point then
* The sum of x and y of the top left corner will be minimum
* Similarly the sum of x and y of the bottom right corner will be maximum
* The difference between x and y will be positive for the top right corner and negative for the bottom left corner
* Using this idea we arrange the coordinates for perspective transform

In [7]:
ordered_points = []
sum_list = []

for x,y in points:
    sum_list.append(x+y)

zipped_lists = zip(sum_list, points)

sorted_zipped_lists = sorted(zipped_lists)

ordered_points = [element for _, element in sorted_zipped_lists]

if ordered_points[1][0] - ordered_points[1][1] < 0:
    temp = ordered_points[1]
    ordered_points[1] = ordered_points[2]
    ordered_points[2] = temp

# Perspective Transfrom

In [8]:
pts1 = np.float32([ [ordered_points[0][0],ordered_points[0][1]]
                   ,[ordered_points[1][0],ordered_points[1][1]]
                   ,[ordered_points[2][0],ordered_points[2][1]]
                   ,[ordered_points[3][0],ordered_points[3][1]]  ])

pts2 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])   

matrix = cv2.getPerspectiveTransform(pts1, pts2) 
non_scanned = cv2.warpPerspective(img2, matrix, (w, h))

# Thresholding for scanning effect

In [9]:
from skimage.filters import threshold_local

scanned = cv2.cvtColor(non_scanned, cv2.COLOR_BGR2GRAY)
T = threshold_local(scanned, 21, offset = 5, method = "gaussian")
scanned = (scanned > T).astype("uint8") * 255

In [10]:
cv2.imshow("Scanned", scanned)
cv2.imshow("Non scanned", non_scanned)
cv2.imshow('image', img)
cv2.imshow('canny', canny)
cv2.imshow('mask', mask)
    

cv2.waitKey(0)
cv2.destroyAllWindows()